In [8]:
# -*- coding: utf-8 -*-
# 说明：
# - 数据：空格分词（word-level），保留 <|endoftext|>，无 PAD（固定长度切块，尾段丢弃）。
# - 模型：PositionEncoding + [Pre-LN→Self-Attn→Residual→Pre-LN→FFN→Residual] × N + 线性输出头。
# - 训练：Lightning，默认混合精度（GPU 上），可设置 max_steps；推荐两阶段（子集→全量）。
# - 推理：greedy 或 top-k + temperature，遇到 <|endoftext|> 或长度上限停止。
#
# 小贴士：
# - 为兼容你现有 training_step 的写法，DataLoader 默认 batch_size=1（可保持最小改动）。
# - Notebook 环境建议 DataLoader 的 num_workers=0，避免多进程兼容性问题。

import os
import json
import math
from collections import Counter
from typing import List, Tuple, Dict, Optional

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from torch.utils.data import TensorDataset, DataLoader

import lightning as L

# 建议启用 Ampere+ 的 Tensor Core 加速（在合适硬件上可提升 matmul 性能）
torch.set_float32_matmul_precision('high')

In [72]:

#####################################
# 数据与分词：空格分词，构建词表与样本
#####################################

def read_text(path: str, subset_bytes: Optional[int] = None) -> str:
    """
    读取 UTF-8 文本。
    - subset_bytes: 若 >0，仅读取前 N 字节（用于“测试版”快速训练验证）。
    """
    with open(path, 'rb') as f:
        data = f.read(subset_bytes) if (subset_bytes is not None and subset_bytes > 0) else f.read()
    return data.decode('utf-8', errors='ignore')


def tokenize_by_space(text: str) -> List[str]:
    """
    空格分词（最简实现）：
    - 直接使用 str.split()；
    - <|endoftext|> 会作为普通 token 保留；
    - 标点会与词同属一个 token（例如 "time,"），便于后续 C 端实现复现。
    """
    return text.lower().split()


def build_vocab(tokens: List[str], vocab_size: int, force_tokens: List[str]) -> Tuple[Dict[str, int], Dict[int, str]]:
    """
    按词频构建词表：
    - force_tokens（如 ["<|endoftext|>", "<UNK>"]）优先放入词表；
    - 其余按频次从高到低加入，直到 vocab_size。
    返回：
    - token_to_id: 词 → id
    - id_to_token: id → 词
    """
    counts = Counter(tokens)
    vocab: List[str] = []
    seen = set()
    # 1) 特殊词优先
    for t in force_tokens:
        if t not in seen:
            vocab.append(t); seen.add(t)
    # 2) 按频次补足
    for tok, _ in counts.most_common():
        if tok in seen:
            continue
        vocab.append(tok); seen.add(tok)
        if len(vocab) >= vocab_size:
            break
    token_to_id = {tok: i for i, tok in enumerate(vocab)}
    id_to_token = {i: tok for tok, i in token_to_id.items()}
    return token_to_id, id_to_token


def encode_tokens(tokens: List[str], token_to_id: Dict[str, int], unk_token: str) -> List[int]:
    """
    将 token 列表编码为 id 序列；未知词映射到 <UNK>。
    """
    unk_id = token_to_id[unk_token]
    return [token_to_id.get(t, unk_id) for t in tokens]


def build_fixed_length_dataset(ids: List[int], seq_len: int) -> Tuple[torch.Tensor, torch.Tensor]:
    """
    固定长度切块（无重叠、无 PAD）：
    - 对长度 L 的 id 序列，取前 floor(L/seq_len)*seq_len 部分并 view 成 [N, seq_len]；
    - 每块 X（长度 seq_len）构造 (inp=X[:-1], lbl=X[1:])，即右移一位；
    - 丢弃不足 seq_len 的尾巴，避免引入 PAD 与复杂 mask。
    返回：
    - inputs: [N, seq_len-1]
    - labels: [N, seq_len-1]
    """
    total = len(ids)
    usable = (total // seq_len) * seq_len
    if usable < seq_len:
        raise ValueError("语料过短：无法构造一个完整的训练块。请增大语料或降低 seq_len。")
    arr = torch.tensor(ids[:usable], dtype=torch.long)
    blocks = arr.view(-1, seq_len)          # [N, seq_len]
    inputs = blocks[:, :-1].contiguous()    # [N, seq_len-1]
    labels = blocks[:, 1:].contiguous()     # [N, seq_len-1]
    return inputs, labels

In [16]:

#####################################
# 模型：位置编码、注意力、FFN、Block、整模
#####################################

class PositionEncoding(nn.Module):
    """
    正弦位置编码（绝对位置）查表：
    - 预先计算 pe[max_len, d_model]；
    - 前向时按序列长度切片与嵌入相加。
    """
    def __init__(self, d_model=128, max_len=512):
        super().__init__()
        pe = torch.zeros(max_len, d_model)                            # [pos, dim]
        position = torch.arange(0, max_len, dtype=torch.float32).unsqueeze(1)  # [pos, 1]
        div_term = torch.exp(torch.arange(0, d_model, 2, dtype=torch.float32) * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)  # 偶数维
        pe[:, 1::2] = torch.cos(position * div_term)  # 奇数维
        self.register_buffer('pe', pe)  # buffer：随模型移动设备，但不参与梯度更新

    def forward(self, x: torch.Tensor):
        """
        x: [seq, d_model] 或 [batch, seq, d_model]
        - 本实现默认 batch_size=1，常用 [seq, d_model]。
        """
        if x.dim() == 2:
            return x + self.pe[:x.size(0), :]
        elif x.dim() == 3:
            seq_len = x.size(1)
            return x + self.pe[:seq_len, :].unsqueeze(0)
        else:
            raise ValueError("PositionEncoding 仅支持 2D 或 3D 张量。")


class Attention(nn.Module):
    """
    单头缩放点积注意力（无偏置 Wq/Wk/Wv，贴近原论文设置）。
    约定输入形状为 [seq, d_model]，便于复用你之前的代码与训练逻辑。
    """
    def __init__(self, d_model=128):
        super().__init__()
        self.W_q = nn.Linear(d_model, d_model, bias=False)
        self.W_k = nn.Linear(d_model, d_model, bias=False)
        self.W_v = nn.Linear(d_model, d_model, bias=False)
        self.row_dim = 0
        self.col_dim = 1

    def forward(self, x: torch.Tensor, mask: Optional[torch.Tensor] = None) -> torch.Tensor:
        # x: [seq, d_model]
        q = self.W_q(x)
        k = self.W_k(x)
        v = self.W_v(x)

        sims = torch.matmul(q, k.transpose(self.row_dim, self.col_dim))     # [seq, seq]
        scaled = sims / (k.size(self.col_dim) ** 0.5)

        if mask is not None:
            # mask: True 表示屏蔽（赋极小值），下三角可见，上三角屏蔽
            masked_value = torch.finfo(scaled.dtype).min
            scaled = scaled.masked_fill(mask, masked_value)


        weights = F.softmax(scaled, dim=self.col_dim)                       # [seq, seq]
        out = torch.matmul(weights, v)                                      # [seq, d_model]
        return out


class FeedForward(nn.Module):
    """
    最小前馈网络（FFN）：Linear(d_model→d_ff) + GELU + Linear(d_ff→d_model)
    """
    def __init__(self, d_model=128, d_ff=512):
        super().__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.act = nn.GELU()
        self.fc2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        return self.fc2(self.act(self.fc1(x)))


class DecoderBlock(nn.Module):
    """
    最小 GPT 块：
    - Pre-LN + Self-Attn + 残差
    - Pre-LN + FFN + 残差
    """
    def __init__(self, d_model=128, d_ff=512):
        super().__init__()
        self.ln1 = nn.LayerNorm(d_model)
        self.attn = Attention(d_model=d_model)
        self.ln2 = nn.LayerNorm(d_model)
        self.ffn = FeedForward(d_model=d_model, d_ff=d_ff)

    def forward(self, x: torch.Tensor, mask: Optional[torch.Tensor] = None):
        a = self.attn(self.ln1(x), mask=mask)
        x = x + a
        f = self.ffn(self.ln2(x))
        x = x + f
        return x


class DecoderOnlyTransformer(L.LightningModule):
    """
    单头、多层堆叠的 Decoder-Only Transformer（适合小故事生成，易于 C 端复现）。
    输入：token_ids [seq]（batch_size=1）
    输出：logits [seq, vocab_size]
    """
    def __init__(self, vocab_size: int, d_model: int = 128, n_layers: int = 2, d_ff: int = 512, max_len: int = 512, lr: float = 3e-4):
        super().__init__()
        self.save_hyperparameters()
        L.seed_everything(42)

        self.we = nn.Embedding(num_embeddings=vocab_size, embedding_dim=d_model)
        self.pe = PositionEncoding(d_model=d_model, max_len=max_len)
        self.blocks = nn.ModuleList([DecoderBlock(d_model=d_model, d_ff=d_ff) for _ in range(n_layers)])
        self.ln_f = nn.LayerNorm(d_model)
        self.head = nn.Linear(d_model, vocab_size)

        self.loss_fn = nn.CrossEntropyLoss()

    def forward(self, token_ids: torch.Tensor) -> torch.Tensor:
        # token_ids: [seq]
        x = self.we(token_ids)     # [seq, d_model]
        x = self.pe(x)             # 叠加位置编码

        # 因果掩码：下三角为可见（False），上三角屏蔽（True）
        seq_len = token_ids.size(0)
        mask = torch.tril(torch.ones((seq_len, seq_len), device=self.device, dtype=torch.bool))
        mask = ~mask  # True 表示屏蔽

        for blk in self.blocks:
            x = blk(x, mask=mask)

        x = self.ln_f(x)
        logits = self.head(x)      # [seq, vocab]
        return logits

    def configure_optimizers(self):
        # 小模型下无需复杂调度器；AdamW + 适度权重衰减足以收敛
        return AdamW(self.parameters(), lr=self.hparams.lr, weight_decay=0.1)

    def training_step(self, batch, batch_idx):
        """
        Notebook 最小改动：保持 batch_size=1，取 batch 中第 0 个样本。
        若未来改为 batch_size>1，需要修改这里去掉 [0] 并适配前向形状。
        """
        inputs, labels = batch          # [1, seq] / [1, seq]
        inputs = inputs[0]              # [seq]
        labels = labels[0]              # [seq]
        logits = self.forward(inputs)   # [seq, vocab]
        loss = self.loss_fn(logits, labels)
        self.log("train_loss", loss, prog_bar=True, on_step=True, on_epoch=True)
        return loss

In [11]:

#####################################
# 采样与推理（greedy / top-k + temperature）
#####################################

def sample_next_id(logits_row: torch.Tensor, temperature: float = 1.0, top_k: Optional[int] = None) -> int:
    """
    从单步 logits（形状 [vocab]）中采样下一个 token id。
    - temperature: >1 更发散，<1 更保守；=1 不变
    - top_k: 仅在前 top_k 里采样（可减少低概率噪声），None/<=0 表示关闭
    """
    if temperature != 1.0:
        logits_row = logits_row / temperature

    if top_k is not None and top_k > 0:
        k = min(top_k, logits_row.size(0))
        top_vals, top_idx = torch.topk(logits_row, k=k)
        filt = torch.full_like(logits_row, float('-inf'))
        filt[top_idx] = top_vals
        logits_row = filt

    probs = F.softmax(logits_row, dim=-1)
    next_id = torch.multinomial(probs, num_samples=1).item()
    return next_id


def generate_text(
    model: DecoderOnlyTransformer,
    token_to_id: Dict[str, int],
    id_to_token: Dict[int, str],
    prompt: str,
    max_new_tokens: int = 256,
    temperature: float = 1.0,
    top_k: Optional[int] = None,
    eos_token: str = "<|endoftext|>"
) -> str:
    """
    自回归生成（batch_size=1）：
    - 默认用采样（temperature/top_k 可控）；若想 greedy，则设置 temperature=1 且 top_k=None，
      并可将 sample_next_id 改成 argmax。
    - 碰到 <|endoftext|> 提前停止，或达到 max_new_tokens 停止。
    """
    unk = "<UNK>"
    model.eval()
    device = next(model.parameters()).device
    with torch.no_grad():
        # 1) 编码 prompt
        prompt_ids = encode_tokens(tokenize_by_space(prompt), token_to_id, unk)
        if len(prompt_ids) == 0:
            return ""

        x = torch.tensor(prompt_ids, dtype=torch.long, device=device)  # [seq0]

        # 2) 逐 token 生成
        for _ in range(max_new_tokens):
            logits = model(x)             # [seq, vocab]
            last = logits[-1]             # 最近一步的 logits
            next_id = sample_next_id(last, temperature=temperature, top_k=top_k)
            x = torch.cat([x, torch.tensor([next_id], dtype=torch.long, device=device)], dim=0)
            if id_to_token.get(next_id, "") == eos_token:
                break

        # 3) 仅返回新增部分（更直观）
        gen_ids = x[len(prompt_ids):]
        tokens = [id_to_token[i.item()] for i in gen_ids]
        return " ".join(tokens)


In [12]:

#####################################
# 训练函数（Notebook 友好）
#####################################

def run_train(
    corpus_path: str,
    save_dir: str,
    subset_mb: int = 3,         # 测试版默认取前 3MB
    vocab_size: int = 4000,
    seq_len: int = 128,
    d_model: int = 128,
    n_layers: int = 2,
    d_ff: int = 512,
    max_len: int = 512,
    lr: float = 3e-4,
    max_steps: int = 500,       # 测试版默认 500 步
    batch_size: int = 1,        # 与 training_step 对齐（最小改动）
    num_workers: int = 0,       # Notebook 建议=0，避免多进程问题
    use_gpu: bool = True,       # 自动选择 GPU/CPU 与精度
):
    """
    训练入口（为 Notebook 设计）：
    - subset_mb>0：仅读取前 N MB 文本用于“测试版”训练；
      subset_mb<=0：读取全量文本用于“全量版”训练。
    - 训练结束保存三类工件：vocab.json、config.json、transformer_state.pt（后续推理与 C 端会用到）。
    """
    os.makedirs(save_dir, exist_ok=True)

    # 1) 读取文本（可选子集）
    subset_bytes = None if (subset_mb is None or subset_mb <= 0) else subset_mb * 1024 * 1024
    text = read_text(corpus_path, subset_bytes=subset_bytes)

    # 2) 分词与词表
    tokens = tokenize_by_space(text)
    force_tokens = ["<|endoftext|>", "<UNK>"]
    token_to_id, id_to_token = build_vocab(tokens, vocab_size=vocab_size, force_tokens=force_tokens)

    # 3) 编码为 id 序列
    ids = encode_tokens(tokens, token_to_id, "<UNK>")

    # 4) 构造固定长度样本（无重叠、无 PAD）
    inputs, labels = build_fixed_length_dataset(ids, seq_len=seq_len)  # 每条样本长度为 seq_len-1
    dataset = TensorDataset(inputs, labels)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=True)

    # 5) 初始化模型
    model = DecoderOnlyTransformer(
        vocab_size=len(token_to_id),
        d_model=d_model,
        n_layers=n_layers,
        d_ff=d_ff,
        max_len=max_len,
        lr=lr
    )

    # 6) Lightning Trainer（自动设备 + 混合精度）
    accelerator = "gpu" if (use_gpu and torch.cuda.is_available()) else "cpu"
    precision = "16-mixed" if accelerator == "gpu" else "32-true"

    trainer = L.Trainer(
        max_steps=max_steps,
        accelerator=accelerator,
        devices=1,
        precision=precision,
        log_every_n_steps=10
    )

    # 7) 开始训练
    trainer.fit(model, train_dataloaders=loader)

    # 8) 保存词表、配置、权重
    with open(os.path.join(save_dir, "vocab.json"), "w", encoding="utf-8") as f:
        json.dump(token_to_id, f, ensure_ascii=False, indent=2)
    config = dict(
        vocab_size=len(token_to_id),
        d_model=d_model,
        n_layers=n_layers,
        d_ff=d_ff,
        max_len=max_len,
        seq_len=seq_len
    )
    with open(os.path.join(save_dir, "config.json"), "w", encoding="utf-8") as f:
        json.dump(config, f, ensure_ascii=False, indent=2)
    torch.save(model.state_dict(), os.path.join(save_dir, "transformer_state.pt"))

    return {
        "save_dir": save_dir,
        "vocab_size": len(token_to_id),
        "num_samples": len(dataset),
        "accelerator": accelerator,
        "precision": precision
    }

In [13]:

#####################################
# 推理函数（Notebook 友好）
#####################################

def run_generate(
    save_dir: str,
    prompt: str,
    temperature: float = 1.0,
    top_k: int = 0,
    max_new_tokens: int = 256,
    use_gpu: bool = False
) -> str:
    """
    推理入口：
    - 从 save_dir 载入 vocab/config/权重；
    - 在 CPU（默认）或 GPU 上构建模型并生成文本。
    返回生成的字符串（不含 prompt，便于观察新增部分）。
    """
    # 1) 加载词表与配置
    with open(os.path.join(save_dir, "vocab.json"), "r", encoding="utf-8") as f:
        token_to_id = json.load(f)
    # 反向字典：id -> token
    id_to_token = {int(v): k for k, v in token_to_id.items()}

    with open(os.path.join(save_dir, "config.json"), "r", encoding="utf-8") as f:
        cfg = json.load(f)

    # 2) 初始化模型并加载权重
    model = DecoderOnlyTransformer(
        vocab_size=cfg["vocab_size"],
        d_model=cfg["d_model"],
        n_layers=cfg["n_layers"],
        d_ff=cfg["d_ff"],
        max_len=cfg["max_len"],
        lr=3e-4
    )

    device = "cuda" if (use_gpu and torch.cuda.is_available()) else "cpu"
    state = torch.load(os.path.join(save_dir, "transformer_state.pt"), map_location=device)
    model.load_state_dict(state)
    model.to(device)
    model.eval()

    # 3) 生成
    out = generate_text(
        model=model,
        token_to_id=token_to_id,
        id_to_token=id_to_token,
        prompt=prompt,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        top_k=(None if top_k <= 0 else top_k),
        eos_token="<|endoftext|>"
    )
    return out

In [81]:
corpus_path = "TinyStories-train.txt"
save_dir = "runs_ipynb/test"
subset_mb = 1024 #仅取前1024MB文本
vocab_size = 32000
seq_len = 512
d_model = 512; n_layers = 4; d_ff = 2048; max_len = 1024
lr = 2e-4; max_steps = 20000
batch_size = 1; num_workers = 0; use_gpu = True

In [82]:
info = run_train(corpus_path, save_dir, subset_mb, vocab_size, seq_len, d_model, n_layers, d_ff, max_len, lr, max_steps, batch_size, num_workers, use_gpu)

Seed set to 42
Using 16bit Automatic Mixed Precision (AMP)
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type             | Params | Mode 
-----------------------------------------------------
0 | we      | Embedding        | 16.4 M | train
1 | pe      | PositionEncoding | 0      | train
2 | blocks  | ModuleList       | 11.6 M | train
3 | ln_f    | LayerNorm        | 1.0 K  | train
4 | head    | Linear           | 16.4 M | train
5 | loss_fn | CrossEntropyLoss | 0      | train
-----------------------------------------------------
44.4 M    Trainable params
0         Non-trainable params
44.4 M    Total params
177.415   Total estimated model params size (MB)
50 

Epoch 0: 100%|██████████| 20000/20000 [08:33<00:00, 38.97it/s, v_num=12, train_loss_step=2.920, train_loss_epoch=3.650]

`Trainer.fit` stopped: `max_steps=20000` reached.


Epoch 0: 100%|██████████| 20000/20000 [08:35<00:00, 38.81it/s, v_num=12, train_loss_step=2.920, train_loss_epoch=3.650]


In [84]:
prompt = "Once upon a time, there was a girl"
print(run_generate(save_dir="runs_ipynb/test", prompt=prompt, temperature=0.9, top_k=50, max_new_tokens=128, use_gpu=False))

Seed set to 42


named amy. amy was three years old. one day, amy was walking through the park with her mom when they saw a swing. amy wanted to get a better look. mama asked her to give the ball away. amy was brave and began to move and she felt happy. the end. <|endoftext|>


usage: ipykernel_launcher.py [-h] --run_dir RUN_DIR
ipykernel_launcher.py: error: the following arguments are required: --run_dir


SystemExit: 2

/home/scheng/github-project/TinyTansformer/.venv/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
